#Importation des librairies et modules
---



In [7]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import sys
import pandas as pd
import numpy as np
import sklearn
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

#Sickit learn met régulièrement à jour des versions et indique des futurs warnings.
#ces deux lignes permettent de ne pas les afficher.
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier


import spacy

from sklearn.decomposition import TruncatedSVD
from imblearn.over_sampling import SMOTE






[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


#Montage du Drive

In [8]:
from google.colab import drive
drive.mount('/content/gdrive/')

my_local_drive='/content/gdrive/My Drive/ProjetML/Dataset'
# Ajout du path pour les librairies, fonctions et données
sys.path.append(my_local_drive)
# Se positionner sur le répertoire associé
%cd $my_local_drive

%pwd

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).
/content/gdrive/My Drive/ProjetML/Dataset


'/content/gdrive/My Drive/ProjetML/Dataset'

#Importation du Dataset avec tous les prétraitements

In [9]:
# Charger les fichiers CSV dans des dataframes pandas
df = pd.read_csv('donnees_pretraitees.csv')

#Visualisation du dataset

In [10]:
display(df.head())

,text,title,our rating,combined,combined_with_lowercase,combined_without_stopwords,combined_with_lemmatization,combined_with_all,rating_true_false,rating_true_false_vs_mixture,rating_all
0,Distracted driving causes more deaths in Canad...,"You Can Be Fined $1,500 If Your Passenger Is U...",false,"You Can Be Fined $1,500 If Your Passenger Is U...","you can be fined $1,500 if your passenger is u...",fined passenger using mobile phone starting ne...,"you can be fine $ 1,500 if your passenger be u...",fine passenger use mobile phone start next wee...,0,0,0
1,Missouri politicians have made statements afte...,Missouri lawmakers condemn Las Vegas shooting,mixture,Missouri lawmakers condemn Las Vegas shooting ...,missouri lawmakers condemn las vegas shooting ...,missouri lawmakers condemn las vegas shooting ...,Missouri lawmaker condemn Las Vegas shoot Miss...,missouri lawmaker condemn las vegas shoot miss...,-1,1,2
2,Home Alone 2: Lost in New York is full of viol...,CBC Cuts Donald Trump's 'Home Alone 2' Cameo O...,mixture,CBC Cuts Donald Trump's 'Home Alone 2' Cameo O...,cbc cuts donald trump's 'home alone 2' cameo o...,cbc cuts donald trump alone cameo broadcast ho...,CBC Cuts Donald Trump 's ' home alone 2 ' Came...,cbc cuts donald trump home alone cameo broadca...,-1,1,2
3,But things took a turn for the worse when riot...,Obama’s Daughters Caught on Camera Burning US ...,false,Obama’s Daughters Caught on Camera Burning US ...,obama’s daughters caught on camera burning us ...,obama daughters caught camera burning us flag ...,Obama ’s daughter catch on Camera burn US Flag...,obama daughter catch camera burn us flag dc pr...,0,0,0
4,It’s no secret that Epstein and Schiff share a...,Leaked Visitor Logs Reveal Schiff’s 78 Visits ...,false,Leaked Visitor Logs Reveal Schiff’s 78 Visits ...,leaked visitor logs reveal schiff’s 78 visits ...,leaked visitor logs reveal schiff visits epste...,Leaked Visitor Logs Reveal Schiff ’s 78 visit ...,leaked visitor logs reveal schiff visit epstei...,0,0,0


#Classification n°2 : (True ou False) Vs. Other

In [11]:
df = df[df['rating_true_false_vs_mixture'] != 1]

# Remplacer les valeurs -1 par 1 dans la colonne 'rating_true_false'
df['rating_true_false_vs_mixture'] = df['rating_true_false_vs_mixture'].replace(-1, 1)

In [53]:
# Compter le nombre de occurrences des valeurs 0 et 1 dans la colonne 'rating_true_false_vs_mixture'
value_counts = df['rating_true_false_vs_mixture'].value_counts()

# Afficher le nombre de occurrences de chaque valeur
print("Nombre de occurrences de la valeur 0 :", value_counts.get(0, 0))
print("Nombre de occurrences de la valeur 1 :", value_counts.get(1, 0))


Nombre de occurrences de la valeur 0 : 1302
Nombre de occurrences de la valeur 1 : 146


#Vectorisation

In [13]:
X_combined = df['combined']
X_without_stopwords = df['combined_without_stopwords']
X_with_lowercase = df['combined_with_lowercase']
X_with_lemmatization = df['combined_with_lemmatization']
X_with_all = df['combined_with_all']

y = df['rating_true_false_vs_mixture']

##TFIDF Victorizer

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Configuration du vectoriseur pour ne pas utiliser les n-grammes
vectorizerTfIdf_unigram = TfidfVectorizer(ngram_range=(1, 1))

# Vectorisation
X_tfidf_combined_unigram = vectorizerTfIdf_unigram.fit_transform(X_combined)
X_tfidf_without_stopwords_unigram = vectorizerTfIdf_unigram.fit_transform(X_without_stopwords)
X_tfidf_with_lowercase_unigram = vectorizerTfIdf_unigram.fit_transform(X_with_lowercase)
X_tfidf_with_lemmatization_unigram = vectorizerTfIdf_unigram.fit_transform(X_with_lemmatization)
X_tfidf_with_all_unigram = vectorizerTfIdf_unigram.fit_transform(X_with_all)


##TfidfVectorizer avec n-grammes

In [15]:
vectorizerTfIdf = TfidfVectorizer(ngram_range=(1, 2))

#vectorization
X_tfidf_combined = vectorizerTfIdf.fit_transform(X_combined)
X_tfidf_without_stopwords = vectorizerTfIdf.fit_transform(X_without_stopwords)
X_tfidf_with_lowercase = vectorizerTfIdf.fit_transform(X_with_lowercase)
X_tfidf_with_lemmatization = vectorizerTfIdf.fit_transform(X_with_lemmatization)
X_tfidf_with_all = vectorizerTfIdf.fit_transform(X_with_all)


##BOW Victorizer

In [16]:
from sklearn.feature_extraction.text import CountVectorizer

# Configuration du vectoriseur pour ne pas utiliser les n-grammes
vectorizerBow_unigram = CountVectorizer(ngram_range=(1, 1))

# Vectorisation
X_bow_combined_unigram = vectorizerBow_unigram.fit_transform(X_combined)
X_bow_without_stopwords_unigram = vectorizerBow_unigram.fit_transform(X_without_stopwords)
X_bow_with_lowercase_unigram = vectorizerBow_unigram.fit_transform(X_with_lowercase)
X_bow_with_lemmatization_unigram = vectorizerBow_unigram.fit_transform(X_with_lemmatization)
X_bow_with_all_unigram = vectorizerBow_unigram.fit_transform(X_with_all)


##BOW victorizer avec ngram

In [17]:
vectorizerBow = CountVectorizer(ngram_range=(1, 2))

#vectorization
X_bow_combined = vectorizerBow.fit_transform(X_combined)
X_bow_without_stopwords = vectorizerBow.fit_transform(X_without_stopwords)
X_bow_with_lowercase = vectorizerBow.fit_transform(X_with_lowercase)
X_bow_with_lemmatization = vectorizerBow.fit_transform(X_with_lemmatization)
X_bow_with_all = vectorizerBow.fit_transform(X_with_all)


#Equilibrage des données

In [18]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from imblearn.over_sampling import SMOTE

# Initialiser SMOTE
smote = SMOTE()


##Equilibrage pour BOW

In [19]:
# Appliquer SMOTE
X_bow_combined_unigram_resampled, y_combined_resampled = smote.fit_resample(X_bow_combined_unigram, y)
X_bow_without_stopwords_unigram_resampled, y_without_stopwords_resampled = smote.fit_resample(X_bow_without_stopwords_unigram, y)
X_bow_with_lowercase_unigram_resampled, y_with_lowercase_resampled = smote.fit_resample(X_bow_with_lowercase_unigram, y)
X_bow_with_lemmatization_unigram_resampled, y_with_lemmatization_resampled = smote.fit_resample(X_bow_with_lemmatization_unigram, y)
X_bow_with_all_unigram_resampled, y_with_all_resampled = smote.fit_resample(X_bow_with_all_unigram, y)

##Equilibrage pour BOW ngrams

In [20]:
smote = SMOTE(random_state=42)

X_bow_combined_resampled, y_combined_resampled = smote.fit_resample(X_bow_combined, y)
X_bow_without_stopwords_resampled, y_without_stopwords_resampled = smote.fit_resample(X_bow_without_stopwords, y)
X_bow_with_lowercase_resampled, y_with_lowercase_resampled = smote.fit_resample(X_bow_with_lowercase, y)
X_bow_with_lemmatization_resampled, y_with_lemmatization_resampled = smote.fit_resample(X_bow_with_lemmatization, y)
X_bow_with_all_resampled, y_with_all_resampled = smote.fit_resample(X_bow_with_all, y)


##Equilibrage pour TDIDF

In [21]:
# Appliquer SMOTE
X_tfidf_combined_unigram_resampled, y_combined_resampled = smote.fit_resample(X_tfidf_combined_unigram, y)
X_tfidf_without_stopwords_unigram_resampled, y_without_stopwords_resampled = smote.fit_resample(X_tfidf_without_stopwords_unigram, y)
X_tfidf_with_lowercase_unigram_resampled, y_with_lowercase_resampled = smote.fit_resample(X_tfidf_with_lowercase_unigram, y)
X_tfidf_with_lemmatization_unigram_resampled, y_with_lemmatization_resampled = smote.fit_resample(X_tfidf_with_lemmatization_unigram, y)
X_tfidf_with_all_unigram_resampled, y_with_all_resampled = smote.fit_resample(X_tfidf_with_all_unigram, y)


##Equilibrage pour TFIDF ngrams

In [22]:
X_tfidf_combined_resampled, y_combined_resampled = smote.fit_resample(X_tfidf_combined, y)
X_tfidf_without_stopwords_resampled, y_without_stopwords_resampled = smote.fit_resample(X_tfidf_without_stopwords, y)
X_tfidf_with_lowercase_resampled, y_with_lowercase_resampled = smote.fit_resample(X_tfidf_with_lowercase, y)
X_tfidf_with_lemmatization_resampled, y_with_lemmatization_resampled = smote.fit_resample(X_tfidf_with_lemmatization, y)
X_tfidf_with_all_resampled, y_with_all_resampled = smote.fit_resample(X_tfidf_with_all, y)

#Split train et test

##Avec le Vectorizer TFIDF

In [23]:
# TF-IDF sans n-grammes - Combined
X_train_tfidf_combined_unigram, X_test_tfidf_combined_unigram, y_train_tfidf_combined_unigram, y_test_tfidf_combined_unigram = train_test_split(X_tfidf_combined_unigram_resampled, y_combined_resampled, test_size=0.2, random_state=42)

# TF-IDF sans n-grammes - Without Stopwords
X_train_tfidf_without_stopwords_unigram, X_test_tfidf_without_stopwords_unigram, y_train_tfidf_without_stopwords_unigram, y_test_tfidf_without_stopwords_unigram = train_test_split(X_tfidf_without_stopwords_unigram_resampled, y_without_stopwords_resampled, test_size=0.2, random_state=42)

# TF-IDF sans n-grammes - With Lowercase
X_train_tfidf_with_lowercase_unigram, X_test_tfidf_with_lowercase_unigram, y_train_tfidf_with_lowercase_unigram, y_test_tfidf_with_lowercase_unigram = train_test_split(X_tfidf_with_lowercase_unigram_resampled, y_with_lowercase_resampled, test_size=0.2, random_state=42)

# TF-IDF sans n-grammes - With Lemmatization
X_train_tfidf_with_lemmatization_unigram, X_test_tfidf_with_lemmatization_unigram, y_train_tfidf_with_lemmatization_unigram, y_test_tfidf_with_lemmatization_unigram = train_test_split(X_tfidf_with_lemmatization_unigram_resampled, y_with_lemmatization_resampled, test_size=0.2, random_state=42)

# TF-IDF sans n-grammes - With All
X_train_tfidf_with_all_unigram, X_test_tfidf_with_all_unigram, y_train_tfidf_with_all_unigram, y_test_tfidf_with_all_unigram = train_test_split(X_tfidf_with_all_unigram_resampled, y_with_all_resampled, test_size=0.2, random_state=42)


##Avec le vectorizer TFIDF ngrams

In [24]:
X_train_tfidf_combined, X_test_tfidf_combined, y_train_tfidf_combined, y_test_tfidf_combined = train_test_split(X_tfidf_combined_resampled, y_combined_resampled, test_size=0.2, random_state=42)
X_train_tfidf_without_stopwords, X_test_tfidf_without_stopwords, y_train_tfidf_without_stopwords, y_test_tfidf_without_stopwords = train_test_split(X_tfidf_without_stopwords_resampled, y_without_stopwords_resampled, test_size=0.2, random_state=42)
X_train_tfidf_with_lowercase, X_test_tfidf_with_lowercase, y_train_tfidf_with_lowercase, y_test_tfidf_with_lowercase = train_test_split(X_tfidf_with_lowercase_resampled, y_with_lowercase_resampled, test_size=0.2, random_state=42)
X_train_tfidf_with_lemmatization, X_test_tfidf_with_lemmatization, y_train_tfidf_with_lemmatization, y_test_tfidf_with_lemmatization = train_test_split(X_tfidf_with_lemmatization_resampled, y_with_lemmatization_resampled, test_size=0.2, random_state=42)
X_train_tfidf_with_all, X_test_tfidf_with_all, y_train_tfidf_with_all, y_test_tfidf_with_all = train_test_split(X_tfidf_with_all_resampled, y_with_all_resampled, test_size=0.2, random_state=42)


##Avec le Vectorizer BOW

In [25]:
# Bag of Words sans n-grammes - Combined
X_train_bow_combined_unigram, X_test_bow_combined_unigram, y_train_bow_combined_unigram, y_test_bow_combined_unigram = train_test_split(X_bow_combined_unigram_resampled, y_combined_resampled, test_size=0.2, random_state=42)

# Bag of Words sans n-grammes - Without Stopwords
X_train_bow_without_stopwords_unigram, X_test_bow_without_stopwords_unigram, y_train_bow_without_stopwords_unigram, y_test_bow_without_stopwords_unigram = train_test_split(X_bow_without_stopwords_unigram_resampled, y_without_stopwords_resampled, test_size=0.2, random_state=42)

# Bag of Words sans n-grammes - With Lowercase
X_train_bow_with_lowercase_unigram, X_test_bow_with_lowercase_unigram, y_train_bow_with_lowercase_unigram, y_test_bow_with_lowercase_unigram = train_test_split(X_bow_with_lowercase_unigram_resampled, y_with_lowercase_resampled, test_size=0.2, random_state=42)

# Bag of Words sans n-grammes - With Lemmatization
X_train_bow_with_lemmatization_unigram, X_test_bow_with_lemmatization_unigram, y_train_bow_with_lemmatization_unigram, y_test_bow_with_lemmatization_unigram = train_test_split(X_bow_with_lemmatization_unigram_resampled, y_with_lemmatization_resampled, test_size=0.2, random_state=42)

# Bag of Words sans n-grammes - With All
X_train_bow_with_all_unigram, X_test_bow_with_all_unigram, y_train_bow_with_all_unigram, y_test_bow_with_all_unigram = train_test_split(X_bow_with_all_unigram_resampled, y_with_all_resampled, test_size=0.2, random_state=42)


##Avec le vectorizer BOW ngrams

In [26]:
X_train_bow_combined, X_test_bow_combined, y_train_bow_combined, y_test_bow_combined = train_test_split(X_bow_combined_resampled, y_combined_resampled, test_size=0.2, random_state=42)
X_train_bow_without_stopwords, X_test_bow_without_stopwords, y_train_bow_without_stopwords, y_test_bow_without_stopwords = train_test_split(X_bow_without_stopwords_resampled, y_without_stopwords_resampled, test_size=0.2, random_state=42)
X_train_bow_with_lowercase, X_test_bow_with_lowercase, y_train_bow_with_lowercase, y_test_bow_with_lowercase = train_test_split(X_bow_with_lowercase_resampled, y_with_lowercase_resampled, test_size=0.2, random_state=42)
X_train_bow_with_lemmatization, X_test_bow_with_lemmatization, y_train_bow_with_lemmatization, y_test_bow_with_lemmatization = train_test_split(X_bow_with_lemmatization_resampled, y_with_lemmatization_resampled, test_size=0.2, random_state=42)
X_train_bow_with_all, X_test_bow_with_all, y_train_bow_with_all, y_test_bow_with_all = train_test_split(X_bow_with_all_resampled, y_with_all_resampled, test_size=0.2, random_state=42)


#Reduction de dimensions

In [27]:
from sklearn.decomposition import TruncatedSVD
n_components_tfidf = 100
svd_tfidf = TruncatedSVD(n_components=n_components_tfidf)

##TFIDF

In [28]:
# Réduction dimensionnelle
X_train_reduced_combined_tfidf = svd_tfidf.fit_transform(X_train_tfidf_combined_unigram)
X_test_reduced_combined_tfidf = svd_tfidf.transform(X_test_tfidf_combined_unigram)

X_train_reduced_without_stopwords_tfidf = svd_tfidf.fit_transform(X_train_tfidf_without_stopwords_unigram)
X_test_reduced_without_stopwords_tfidf = svd_tfidf.transform(X_test_tfidf_without_stopwords_unigram)

X_train_reduced_with_lowercase_tfidf = svd_tfidf.fit_transform(X_train_tfidf_with_lowercase_unigram)
X_test_reduced_with_lowercase_tfidf = svd_tfidf.transform(X_test_tfidf_with_lowercase_unigram)

X_train_reduced_with_lemmatization_tfidf = svd_tfidf.fit_transform(X_train_tfidf_with_lemmatization_unigram)
X_test_reduced_with_lemmatization_tfidf = svd_tfidf.transform(X_test_tfidf_with_lemmatization_unigram)

X_train_reduced_with_all_tfidf = svd_tfidf.fit_transform(X_train_tfidf_with_all_unigram)
X_test_reduced_with_all_tfidf = svd_tfidf.transform(X_test_tfidf_with_all_unigram)



##TFIDF ngrams

In [29]:
X_train_reduced_combined = svd_tfidf.fit_transform(X_train_bow_combined)
X_test_reduced_combined = svd_tfidf.transform(X_test_bow_combined)

X_train_reduced_without_stopwords = svd_tfidf.fit_transform(X_train_bow_without_stopwords)
X_test_reduced_without_stopwords = svd_tfidf.transform(X_test_bow_without_stopwords)

X_train_reduced_with_lowercase = svd_tfidf.fit_transform(X_train_bow_with_lowercase)
X_test_reduced_with_lowercase = svd_tfidf.transform(X_test_bow_with_lowercase)

X_train_reduced_with_lemmatization = svd_tfidf.fit_transform(X_train_bow_with_lemmatization)
X_test_reduced_with_lemmatization = svd_tfidf.transform(X_test_bow_with_lemmatization)

X_train_reduced_with_all = svd_tfidf.fit_transform(X_train_bow_with_all)
X_test_reduced_with_all = svd_tfidf.transform(X_test_bow_with_all)

##BOW

In [30]:
# Réduction dimensionnelle
X_train_reduced_combined_bow = svd_tfidf.fit_transform(X_train_bow_combined_unigram)
X_test_reduced_combined_bow = svd_tfidf.transform(X_test_bow_combined_unigram)

X_train_reduced_without_stopwords_bow = svd_tfidf.fit_transform(X_train_bow_without_stopwords_unigram)
X_test_reduced_without_stopwords_bow = svd_tfidf.transform(X_test_bow_without_stopwords_unigram)

X_train_reduced_with_lowercase_bow = svd_tfidf.fit_transform(X_train_bow_with_lowercase_unigram)
X_test_reduced_with_lowercase_bow = svd_tfidf.transform(X_test_bow_with_lowercase_unigram)

X_train_reduced_with_lemmatization_bow = svd_tfidf.fit_transform(X_train_bow_with_lemmatization_unigram)
X_test_reduced_with_lemmatization_bow = svd_tfidf.transform(X_test_bow_with_lemmatization_unigram)

X_train_reduced_with_all_bow = svd_tfidf.fit_transform(X_train_bow_with_all_unigram)
X_test_reduced_with_all_bow = svd_tfidf.transform(X_test_bow_with_all_unigram)


##BOW ngrams

In [31]:
X_train_reduced_combined = svd_tfidf.fit_transform(X_train_bow_combined)
X_test_reduced_combined = svd_tfidf.transform(X_test_bow_combined)

X_train_reduced_without_stopwords = svd_tfidf.fit_transform(X_train_bow_without_stopwords)
X_test_reduced_without_stopwords = svd_tfidf.transform(X_test_bow_without_stopwords)

X_train_reduced_with_lowercase = svd_tfidf.fit_transform(X_train_bow_with_lowercase)
X_test_reduced_with_lowercase = svd_tfidf.transform(X_test_bow_with_lowercase)

X_train_reduced_with_lemmatization = svd_tfidf.fit_transform(X_train_bow_with_lemmatization)
X_test_reduced_with_lemmatization = svd_tfidf.transform(X_test_bow_with_lemmatization)

X_train_reduced_with_all = svd_tfidf.fit_transform(X_train_bow_with_all)
X_test_reduced_with_all = svd_tfidf.transform(X_test_bow_with_all)

#Classification avec le modèle LogesticRegression

##Cherche hyperparamètres avec BOW ngrams

In [32]:
model = LogisticRegression(max_iter=1000)
solvers = ['lbfgs', 'liblinear','newton-cg']
penalty = ['l2','none']
c_values = [10, 1.0, 0.1]

grid = dict(solver=solvers, penalty=penalty, C=c_values)

grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=[(slice(None), slice(None))], scoring='accuracy', error_score=0)
grid_result = grid_search.fit(X_train_bow_combined, y_train_bow_combined)

print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")

best_params_bow = grid_result.best_params_

Best: 1.0 using {'C': 10, 'penalty': 'l2', 'solver': 'lbfgs'}


##Chercher hyperparametres avec BOW

In [33]:
model = LogisticRegression(max_iter=1000)
solvers = ['lbfgs', 'liblinear','newton-cg']
penalty = ['l2','none']
c_values = [10, 1.0, 0.1]

grid = dict(solver=solvers, penalty=penalty, C=c_values)

grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=[(slice(None), slice(None))], scoring='accuracy', error_score=0)
grid_result = grid_search.fit(X_train_bow_combined_unigram, y_train_bow_combined_unigram)

print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")

best_params_bow_unigram = grid_result.best_params_

Best: 1.0 using {'C': 10, 'penalty': 'l2', 'solver': 'lbfgs'}


##Chercher les hyperparamètres pour TFIDF

In [34]:
model = LogisticRegression(max_iter=1000)
solvers = ['lbfgs', 'liblinear','newton-cg']
penalty = ['l2','none']
c_values = [10, 1.0, 0.1]

grid = dict(solver=solvers, penalty=penalty, C=c_values)

grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=[(slice(None), slice(None))], scoring='accuracy', error_score=0)
grid_result_combined_unigram = grid_search.fit(X_train_tfidf_combined_unigram, y_train_tfidf_combined_unigram)

grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=[(slice(None), slice(None))], scoring='accuracy', error_score=0)
grid_result_all = grid_search.fit(X_train_tfidf_with_all_unigram, y_train_tfidf_with_all_unigram)

print(f"Best combined: {grid_result_combined_unigram.best_score_} using {grid_result_combined_unigram.best_params_}")
print(f"Best with all: {grid_result_all.best_score_} using {grid_result_all.best_params_}")


best_params_tfidf_combined_unigram = grid_result_combined_unigram.best_params_
best_params_tfidf_all_unigram = grid_result_all.best_params_



Best combined: 1.0 using {'C': 10, 'penalty': 'none', 'solver': 'lbfgs'}
Best with all: 1.0 using {'C': 10, 'penalty': 'none', 'solver': 'lbfgs'}


##Chercher les hyperparametrès pour TFIDF ngrams

In [35]:
model = LogisticRegression(max_iter=1000)
solvers = ['lbfgs', 'liblinear','newton-cg']
penalty = ['l2','none']
c_values = [10, 1.0, 0.1]

grid = dict(solver=solvers, penalty=penalty, C=c_values)

grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=[(slice(None), slice(None))], scoring='accuracy', error_score=0)
grid_result_combined = grid_search.fit(X_train_tfidf_combined, y_train_tfidf_combined)

grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=[(slice(None), slice(None))], scoring='accuracy', error_score=0)
grid_result_all = grid_search.fit(X_train_tfidf_with_all, y_train_tfidf_with_all)

print(f"Best combined: {grid_result_combined.best_score_} using {grid_result_combined.best_params_}")
print(f"Best with all: {grid_result_all.best_score_} using {grid_result_all.best_params_}")


best_params_tfidf_combined = grid_result_combined.best_params_
best_params_tfidf_all = grid_result_all.best_params_



Best combined: 1.0 using {'C': 10, 'penalty': 'none', 'solver': 'lbfgs'}
Best with all: 1.0 using {'C': 10, 'penalty': 'none', 'solver': 'lbfgs'}


#Evaluation LogisticRegression

##Test des resultats avec les différents prétraitements avec la vectorisation tf-idf ngrams


In [46]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, classification_report

best_params_tfidf = best_params_tfidf_all

def evaluate_model(X_train, y_train, X_test, y_test):
    model = LogisticRegression(**best_params_tfidf, max_iter=1000)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    cv_results = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy')
    report = classification_report(y_test, y_pred, output_dict=True)

    return accuracy, cv_results, report

datasets = {
    "combined": (X_train_tfidf_combined, y_train_tfidf_combined, X_test_tfidf_combined, y_test_tfidf_combined),
    "without_stopwords": (X_train_tfidf_without_stopwords, y_train_tfidf_without_stopwords, X_test_tfidf_without_stopwords, y_test_tfidf_without_stopwords),
    "with_lowercase": (X_train_tfidf_with_lowercase, y_train_tfidf_with_lowercase, X_test_tfidf_with_lowercase, y_test_tfidf_with_lowercase),
    "with_lemmatization": (X_train_tfidf_with_lemmatization, y_train_tfidf_with_lemmatization, X_test_tfidf_with_lemmatization, y_test_tfidf_with_lemmatization),
    "with_all": (X_train_tfidf_with_all, y_train_tfidf_with_all, X_test_tfidf_with_all, y_test_tfidf_with_all)
}

results = {}
for name, (X_train, y_train, X_test, y_test) in datasets.items():
    accuracy, cv_results, report = evaluate_model(X_train, y_train, X_test, y_test)
    results[name] = {
        "accuracy": accuracy,
        "cv_results": cv_results,
        "classification_report": report
    }
    print(f"Dataset: {name}, Test Accuracy: {accuracy}, Cross-Validation Mean Accuracy: {cv_results.mean()}, Std: {cv_results.std()}")
    print(f"Classification Report for {name}:\n{pd.DataFrame(report).transpose()}\n")

# Afficher les résultats sous forme de DataFrame
results_df = pd.DataFrame.from_dict(results, orient='index')
print(results_df)


Dataset: combined, Test Accuracy: 0.980806142034549, Cross-Validation Mean Accuracy: 0.9836711400110681, Std: 0.01001268341251448
Classification Report for combined:
              precision    recall  f1-score     support
0              0.988462  0.973485  0.980916  264.000000
1              0.973180  0.988327  0.980695  257.000000
accuracy       0.980806  0.980806  0.980806    0.980806
macro avg      0.980821  0.980906  0.980806  521.000000
weighted avg   0.980923  0.980806  0.980807  521.000000

Dataset: without_stopwords, Test Accuracy: 0.982725527831094, Cross-Validation Mean Accuracy: 0.9903938387751339, Std: 0.006085863841312969
Classification Report for without_stopwords:
              precision    recall  f1-score     support
0              1.000000  0.965909  0.982659  264.000000
1              0.966165  1.000000  0.982792  257.000000
accuracy       0.982726  0.982726  0.982726    0.982726
macro avg      0.983083  0.982955  0.982725  521.000000
weighted avg   0.983310  0.98272

In [62]:
for dataset, info in results.items():
    print(f"Dataset: {dataset}")
    classification_report = info['classification_report']
    for class_label, class_info in classification_report.items():
        if class_label.isdigit():
            support = class_info['support']
            print(f"Classe {class_label}: {support} lignes")


Dataset: combined
Classe 0: 264 lignes
Classe 1: 257 lignes
Dataset: without_stopwords
Classe 0: 264 lignes
Classe 1: 257 lignes
Dataset: with_lowercase
Classe 0: 264 lignes
Classe 1: 257 lignes
Dataset: with_lemmatization
Classe 0: 264 lignes
Classe 1: 257 lignes
Dataset: with_all
Classe 0: 264 lignes
Classe 1: 257 lignes



##Test des resultats avec les différents prétraitements avec la vectorisation tf-idf


In [47]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, classification_report

best_params_tfidf_unigram = best_params_tfidf_all_unigram

def evaluate_model(X_train, y_train, X_test, y_test):
    model = LogisticRegression(**best_params_tfidf_unigram, max_iter=1000)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    cv_results = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy')
    report = classification_report(y_test, y_pred, output_dict=True)

    return accuracy, cv_results, report

datasets = {
    "combined": (X_train_tfidf_combined_unigram, y_train_tfidf_combined_unigram, X_test_tfidf_combined_unigram, y_test_tfidf_combined_unigram),
    "without_stopwords": (X_train_tfidf_without_stopwords_unigram, y_train_tfidf_without_stopwords_unigram, X_test_tfidf_without_stopwords_unigram, y_test_tfidf_without_stopwords_unigram),
    "with_lowercase": (X_train_tfidf_with_lowercase_unigram, y_train_tfidf_with_lowercase_unigram, X_test_tfidf_with_lowercase_unigram, y_test_tfidf_with_lowercase_unigram),
    "with_lemmatization": (X_train_tfidf_with_lemmatization_unigram, y_train_tfidf_with_lemmatization_unigram, X_test_tfidf_with_lemmatization_unigram, y_test_tfidf_with_lemmatization_unigram),
    "with_all": (X_train_tfidf_with_all_unigram, y_train_tfidf_with_all_unigram, X_test_tfidf_with_all_unigram, y_test_tfidf_with_all_unigram)
}

results = {}
for name, (X_train, y_train, X_test, y_test) in datasets.items():
    accuracy, cv_results, report = evaluate_model(X_train, y_train, X_test, y_test)
    results[name] = {
        "accuracy": accuracy,
        "cv_results": cv_results,
        "classification_report": report
    }
    print(f"Dataset: {name}, Test Accuracy: {accuracy}, Cross-Validation Mean Accuracy: {cv_results.mean()}, Std: {cv_results.std()}")
    print(f"Classification Report for {name}:\n{pd.DataFrame(report).transpose()}\n")

# Afficher les résultats sous forme de DataFrame
results_df = pd.DataFrame.from_dict(results, orient='index')
print(results_df)


Dataset: combined, Test Accuracy: 0.9865642994241842, Cross-Validation Mean Accuracy: 0.9716645914038, Std: 0.011929943783382415
Classification Report for combined:
              precision    recall  f1-score     support
0              1.000000  0.973485  0.986564  264.000000
1              0.973485  1.000000  0.986564  257.000000
accuracy       0.986564  0.986564  0.986564    0.986564
macro avg      0.986742  0.986742  0.986564  521.000000
weighted avg   0.986921  0.986564  0.986564  521.000000

Dataset: without_stopwords, Test Accuracy: 0.9884836852207294, Cross-Validation Mean Accuracy: 0.9755049806308799, Std: 0.014461860799979478
Classification Report for without_stopwords:
              precision    recall  f1-score     support
0              1.000000  0.977273  0.988506  264.000000
1              0.977186  1.000000  0.988462  257.000000
accuracy       0.988484  0.988484  0.988484    0.988484
macro avg      0.988593  0.988636  0.988484  521.000000
weighted avg   0.988746  0.98848

##Test des resultats avec les différents prétraitements avec la vectorisation BOW

In [48]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, classification_report

best_params_bow_unigram = best_params_bow_unigram

def evaluate_model_bow(X_train, y_train, X_test, y_test):
    model = LogisticRegression(**best_params_bow_unigram, max_iter=1000)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    cv_results = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy')
    report = classification_report(y_test, y_pred, output_dict=True)

    return accuracy, cv_results, report

datasets_bow = {
    "combined": (X_train_bow_combined_unigram, y_train_bow_combined_unigram, X_test_bow_combined_unigram, y_test_bow_combined_unigram),
    "without_stopwords": (X_train_bow_without_stopwords_unigram, y_train_bow_without_stopwords_unigram, X_test_bow_without_stopwords_unigram, y_test_bow_without_stopwords_unigram),
    "with_lowercase": (X_train_bow_with_lowercase_unigram, y_train_bow_with_lowercase_unigram, X_test_bow_with_lowercase_unigram, y_test_bow_with_lowercase_unigram),
    "with_lemmatization": (X_train_bow_with_lemmatization_unigram, y_train_bow_with_lemmatization_unigram, X_test_bow_with_lemmatization_unigram, y_test_bow_with_lemmatization_unigram),
    "with_all": (X_train_bow_with_all_unigram, y_train_bow_with_all_unigram, X_test_bow_with_all_unigram, y_test_bow_with_all_unigram)
}

results_bow = {}
for name, (X_train, y_train, X_test, y_test) in datasets_bow.items():
    accuracy, cv_results, report = evaluate_model_bow(X_train, y_train, X_test, y_test)
    results_bow[name] = {
        "accuracy": accuracy,
        "cv_results": cv_results,
        "classification_report": report
    }
    print(f"Dataset BoW: {name}, Test Accuracy: {accuracy}, Cross-Validation Mean Accuracy: {cv_results.mean()}, Std: {cv_results.std()}")
    print(f"Classification Report for BoW {name}:\n{pd.DataFrame(report).transpose()}\n")

# Afficher les résultats sous forme de DataFrame
results_bow_df = pd.DataFrame.from_dict(results_bow, orient='index')
print(results_bow_df)


Dataset BoW: combined, Test Accuracy: 0.9251439539347409, Cross-Validation Mean Accuracy: 0.9202937649880095, Std: 0.014108881524445518
Classification Report for BoW combined:
              precision    recall  f1-score     support
0              0.959184  0.890152  0.923379  264.000000
1              0.894928  0.961089  0.926829  257.000000
accuracy       0.925144  0.925144  0.925144    0.925144
macro avg      0.927056  0.925621  0.925104  521.000000
weighted avg   0.927487  0.925144  0.925081  521.000000

Dataset BoW: without_stopwords, Test Accuracy: 0.927063339731286, Cross-Validation Mean Accuracy: 0.9159829828444938, Std: 0.006667668126220773
Classification Report for BoW without_stopwords:
              precision    recall  f1-score     support
0              0.963115  0.890152  0.925197  264.000000
1              0.895307  0.964981  0.928839  257.000000
accuracy       0.927063  0.927063  0.927063    0.927063
macro avg      0.929211  0.927566  0.927018  521.000000
weighted avg  

##Test des resultats avec les différents prétraitements avec la vectorisation BOW ngrams

In [50]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, classification_report

best_params_bow = best_params_bow

def evaluate_model_bow(X_train, y_train, X_test, y_test):
    model = LogisticRegression(**best_params_bow, max_iter=1000)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    cv_results = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy')
    report = classification_report(y_test, y_pred, output_dict=True)

    return accuracy, cv_results, report

datasets_bow = {
    "combined": (X_train_bow_combined, y_train_bow_combined, X_test_bow_combined, y_test_bow_combined),
    "without_stopwords": (X_train_bow_without_stopwords, y_train_bow_without_stopwords, X_test_bow_without_stopwords, y_test_bow_without_stopwords),
    "with_lowercase": (X_train_bow_with_lowercase, y_train_bow_with_lowercase, X_test_bow_with_lowercase, y_test_bow_with_lowercase),
    "with_lemmatization": (X_train_bow_with_lemmatization, y_train_bow_with_lemmatization, X_test_bow_with_lemmatization, y_test_bow_with_lemmatization),
    "with_all": (X_train_bow_with_all, y_train_bow_with_all, X_test_bow_with_all, y_test_bow_with_all)
}

results_bow = {}
for name, (X_train, y_train, X_test, y_test) in datasets_bow.items():
    accuracy, cv_results, report = evaluate_model_bow(X_train, y_train, X_test, y_test)
    results_bow[name] = {
        "accuracy": accuracy,
        "cv_results": cv_results,
        "classification_report": report
    }
    print(f"Dataset BoW: {name}, Test Accuracy: {accuracy}, Cross-Validation Mean Accuracy: {cv_results.mean()}, Std: {cv_results.std()}")
    print(f"Classification Report for BoW {name}:\n{pd.DataFrame(report).transpose()}\n")

# Afficher les résultats sous forme de DataFrame
results_bow_df = pd.DataFrame.from_dict(results_bow, orient='index')
print(results_bow_df)


Dataset BoW: combined, Test Accuracy: 0.9366602687140115, Cross-Validation Mean Accuracy: 0.9241422246817932, Std: 0.011059249084018131
Classification Report for BoW combined:
              precision    recall  f1-score    support
0              0.963855  0.909091  0.935673  264.00000
1              0.911765  0.964981  0.937618  257.00000
accuracy       0.936660  0.936660  0.936660    0.93666
macro avg      0.937810  0.937036  0.936645  521.00000
weighted avg   0.938160  0.936660  0.936632  521.00000

Dataset BoW: without_stopwords, Test Accuracy: 0.9347408829174664, Cross-Validation Mean Accuracy: 0.9227022228371149, Std: 0.0060316782596148
Classification Report for BoW without_stopwords:
              precision    recall  f1-score     support
0              0.971311  0.897727  0.933071  264.000000
1              0.902527  0.972763  0.936330  257.000000
accuracy       0.934741  0.934741  0.934741    0.934741
macro avg      0.936919  0.935245  0.934700  521.000000
weighted avg   0.9373

#Classification avec le modèle Random Forest

##Chercher les best hyperparameters pour Random Forest avec BOW ngrams

In [65]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

model = RandomForestClassifier()

# Définir la grille des hyperparamètres
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

# Créer l'objet GridSearchCV
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, n_jobs=-1, scoring='accuracy')

# Effectuer la recherche sur la grille
grid_result = grid_search.fit(X_train_bow_combined, y_train_bow_combined)

# Afficher les meilleurs résultats
print(f"Best accuracy: {grid_result.best_score_} using {grid_result.best_params_}")

best_params_bow_combined = grid_result.best_params_


Best accuracy: 0.9279849197565024 using {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 200}


##Chercher les best hyperparameters pour Random Forest avec BOW

In [66]:
# Créer l'objet GridSearchCV
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, n_jobs=-1, scoring='accuracy')

# Effectuer la recherche sur la grille
grid_result = grid_search.fit(X_train_bow_combined_unigram, y_train_bow_combined_unigram)

# Afficher les meilleurs résultats
print(f"Best accuracy: {grid_result.best_score_} using {grid_result.best_params_}")

best_params_bow_combined_unigram = grid_result.best_params_

Best accuracy: 0.9515057185021213 using {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}


##Chercher les best hyperparameters pour Random Forest avec TFIDF ngrams

In [68]:
# Créer l'objet GridSearchCV
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, n_jobs=-1, scoring='accuracy')

# Effectuer la recherche sur la grille
grid_result = grid_search.fit(X_train_tfidf_combined, y_train_tfidf_combined)

# Afficher les meilleurs résultats
print(f"Best accuracy: {grid_result.best_score_} using {grid_result.best_params_}")

best_params_tfidf_combined = grid_result.best_params_

Best accuracy: 0.9663945766463751 using {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}


##Chercher les best hyperparameters pour Random Forest avec TFIDF ngrams

In [69]:
# Créer l'objet GridSearchCV
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, n_jobs=-1, scoring='accuracy')

# Effectuer la recherche sur la grille
grid_result = grid_search.fit(X_train_tfidf_combined_unigram, y_train_tfidf_combined_unigram)

# Afficher les meilleurs résultats
print(f"Best accuracy: {grid_result.best_score_} using {grid_result.best_params_}")

best_params_tfidf_combined_unigram = grid_result.best_params_

Best accuracy: 0.9654238147943184 using {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}


#Evaluation Random Forest

##Test des resultats avec les différents prétraitements avec la vectorisation tf-idf


In [70]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, classification_report

def evaluate_model(X_train, y_train, X_test, y_test, params):
    model = RandomForestClassifier(**params)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    cv_results = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy')
    report = classification_report(y_test, y_pred, output_dict=True)

    return accuracy, cv_results, report

best_params = best_params_tfidf_combined_unigram

datasets_tfidf = {
    "combined": (X_train_tfidf_combined_unigram, y_train_tfidf_combined_unigram, X_test_tfidf_combined_unigram, y_test_tfidf_combined_unigram),
    "without_stopwords": (X_train_tfidf_without_stopwords_unigram, y_train_tfidf_without_stopwords_unigram, X_test_tfidf_without_stopwords_unigram, y_test_tfidf_without_stopwords_unigram),
    "with_lowercase": (X_train_tfidf_with_lowercase_unigram, y_train_tfidf_with_lowercase_unigram, X_test_tfidf_with_lowercase_unigram, y_test_tfidf_with_lowercase_unigram),
    "with_lemmatization": (X_train_tfidf_with_lemmatization_unigram, y_train_tfidf_with_lemmatization_unigram, X_test_tfidf_with_lemmatization_unigram, y_test_tfidf_with_lemmatization_unigram),
    "with_all": (X_train_tfidf_with_all_unigram, y_train_tfidf_with_all_unigram, X_test_tfidf_with_all_unigram, y_test_tfidf_with_all_unigram)
}

results_tfidf = {}
for name, (X_train, y_train, X_test, y_test) in datasets_tfidf.items():
    accuracy, cv_results, report = evaluate_model(X_train, y_train, X_test, y_test, best_params)
    results_tfidf[name] = {
        "accuracy": accuracy,
        "cv_results": cv_results,
        "classification_report": report
    }
    print(f"TF-IDF Dataset: {name}, Test Accuracy: {accuracy}, Cross-Validation Mean Accuracy: {cv_results.mean()}, Std: {cv_results.std()}")
    print(f"Classification Report for TF-IDF {name}:\n{pd.DataFrame(report).transpose()}\n")

# Afficher les résultats sous forme de DataFrame
results_tfidf_df = pd.DataFrame.from_dict(results_tfidf, orient='index')
print(results_tfidf_df)


TF-IDF Dataset: combined, Test Accuracy: 0.9769673704414588, Cross-Validation Mean Accuracy: 0.9659057369489025, Std: 0.01229633413677116
Classification Report for TF-IDF combined:
              precision    recall  f1-score     support
0              0.963235  0.992424  0.977612  264.000000
1              0.991968  0.961089  0.976285  257.000000
accuracy       0.976967  0.976967  0.976967    0.976967
macro avg      0.977602  0.976757  0.976948  521.000000
weighted avg   0.977409  0.976967  0.976957  521.000000

TF-IDF Dataset: without_stopwords, Test Accuracy: 0.982725527831094, Cross-Validation Mean Accuracy: 0.9755107452499538, Std: 0.0065305297215774065
Classification Report for TF-IDF without_stopwords:
              precision    recall  f1-score     support
0              0.981132  0.984848  0.982987  264.000000
1              0.984375  0.980545  0.982456  257.000000
accuracy       0.982726  0.982726  0.982726    0.982726
macro avg      0.982754  0.982697  0.982721  521.000000
we

##Test des resultats avec les différents prétraitements avec la vectorisation tf-idf ngrams


In [72]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, classification_report

def evaluate_model(X_train, y_train, X_test, y_test, params):
    model = RandomForestClassifier(**params)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    cv_results = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy')
    report = classification_report(y_test, y_pred, output_dict=True)

    return accuracy, cv_results, report

best_params = best_params_tfidf_combined

datasets_tfidf = {
    "combined": (X_train_tfidf_combined, y_train_tfidf_combined, X_test_tfidf_combined, y_test_tfidf_combined),
    "without_stopwords": (X_train_tfidf_without_stopwords, y_train_tfidf_without_stopwords, X_test_tfidf_without_stopwords, y_test_tfidf_without_stopwords),
    "with_lowercase": (X_train_tfidf_with_lowercase, y_train_tfidf_with_lowercase, X_test_tfidf_with_lowercase, y_test_tfidf_with_lowercase),
    "with_lemmatization": (X_train_tfidf_with_lemmatization, y_train_tfidf_with_lemmatization, X_test_tfidf_with_lemmatization, y_test_tfidf_with_lemmatization),
    "with_all": (X_train_tfidf_with_all, y_train_tfidf_with_all, X_test_tfidf_with_all, y_test_tfidf_with_all)
}

results_tfidf = {}
for name, (X_train, y_train, X_test, y_test) in datasets_tfidf.items():
    accuracy, cv_results, report = evaluate_model(X_train, y_train, X_test, y_test, best_params)
    results_tfidf[name] = {
        "accuracy": accuracy,
        "cv_results": cv_results,
        "classification_report": report
    }
    print(f"TF-IDF Dataset: {name}, Test Accuracy: {accuracy}, Cross-Validation Mean Accuracy: {cv_results.mean()}, Std: {cv_results.std()}")
    print(f"Classification Report for TF-IDF {name}:\n{pd.DataFrame(report).transpose()}\n")

# Afficher les résultats sous forme de DataFrame
results_tfidf_df = pd.DataFrame.from_dict(results_tfidf, orient='index')
print(results_tfidf_df)


TF-IDF Dataset: combined, Test Accuracy: 0.9673704414587332, Cross-Validation Mean Accuracy: 0.9663934237225604, Std: 0.006260194136980229
Classification Report for TF-IDF combined:
              precision    recall  f1-score    support
0              0.945848  0.992424  0.968577  264.00000
1              0.991803  0.941634  0.966068  257.00000
accuracy       0.967370  0.967370  0.967370    0.96737
macro avg      0.968826  0.967029  0.967322  521.00000
weighted avg   0.968517  0.967370  0.967339  521.00000

TF-IDF Dataset: without_stopwords, Test Accuracy: 0.9846449136276392, Cross-Validation Mean Accuracy: 0.9822345969378343, Std: 0.006198781924495952
Classification Report for TF-IDF without_stopwords:
              precision    recall  f1-score     support
0              0.977612  0.992424  0.984962  264.000000
1              0.992095  0.976654  0.984314  257.000000
accuracy       0.984645  0.984645  0.984645    0.984645
macro avg      0.984853  0.984539  0.984638  521.000000
weighte

##Test des resultats avec les différents prétraitements avec la vectorisation BOW


In [73]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, classification_report

def evaluate_model(X_train, y_train, X_test, y_test, params):
    model = RandomForestClassifier(**params)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    cv_results = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy')
    report = classification_report(y_test, y_pred, output_dict=True)

    return accuracy, cv_results, report

best_params = best_params_bow_combined_unigram

datasets_bow = {
    "combined": (X_train_bow_combined_unigram, y_train_bow_combined_unigram, X_test_bow_combined_unigram, y_test_bow_combined_unigram),
    "without_stopwords": (X_train_bow_without_stopwords_unigram, y_train_bow_without_stopwords_unigram, X_test_bow_without_stopwords_unigram, y_test_bow_without_stopwords_unigram),
    "with_lowercase": (X_train_bow_with_lowercase_unigram, y_train_bow_with_lowercase_unigram, X_test_bow_with_lowercase_unigram, y_test_bow_with_lowercase_unigram),
    "with_lemmatization": (X_train_bow_with_lemmatization_unigram, y_train_bow_with_lemmatization_unigram, X_test_bow_with_lemmatization_unigram, y_test_bow_with_lemmatization_unigram),
    "with_all": (X_train_bow_with_all_unigram, y_train_bow_with_all_unigram, X_test_bow_with_all_unigram, y_test_bow_with_all_unigram)
}

results_bow = {}
for name, (X_train, y_train, X_test, y_test) in datasets_bow.items():
    accuracy, cv_results, report = evaluate_model(X_train, y_train, X_test, y_test, best_params)
    results_bow[name] = {
        "accuracy": accuracy,
        "cv_results": cv_results,
        "classification_report": report
    }
    print(f"BoW Dataset: {name}, Test Accuracy: {accuracy}, Cross-Validation Mean Accuracy: {cv_results.mean()}, Std: {cv_results.std()}")
    print(f"Classification Report for BoW {name}:\n{pd.DataFrame(report).transpose()}\n")

# Afficher les résultats sous forme de DataFrame
results_bow_df = pd.DataFrame.from_dict(results_bow, orient='index')
print(results_bow_df)


BoW Dataset: combined, Test Accuracy: 0.963531669865643, Cross-Validation Mean Accuracy: 0.951506871425936, Std: 0.009673593097796881
Classification Report for BoW combined:
              precision    recall  f1-score     support
0              0.962264  0.965909  0.964083  264.000000
1              0.964844  0.961089  0.962963  257.000000
accuracy       0.963532  0.963532  0.963532    0.963532
macro avg      0.963554  0.963499  0.963523  521.000000
weighted avg   0.963537  0.963532  0.963531  521.000000

BoW Dataset: without_stopwords, Test Accuracy: 0.9500959692898272, Cross-Validation Mean Accuracy: 0.934221084670725, Std: 0.008990710589631483
Classification Report for BoW without_stopwords:
              precision    recall  f1-score     support
0              0.947368  0.954545  0.950943  264.000000
1              0.952941  0.945525  0.949219  257.000000
accuracy       0.950096  0.950096  0.950096    0.950096
macro avg      0.950155  0.950035  0.950081  521.000000
weighted avg   0

##Test des resultats avec les différents prétraitements avec la vectorisation BOW ngrams


In [74]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, classification_report

def evaluate_model(X_train, y_train, X_test, y_test, params):
    model = RandomForestClassifier(**params)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    cv_results = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy')
    report = classification_report(y_test, y_pred)

    return accuracy, cv_results, report

best_params = best_params_bow_combined

datasets_bow = {
    "combined": (X_train_bow_combined, y_train_bow_combined, X_test_bow_combined, y_test_bow_combined),
    "without_stopwords": (X_train_bow_without_stopwords, y_train_bow_without_stopwords, X_test_bow_without_stopwords, y_test_bow_without_stopwords),
    "with_lowercase": (X_train_bow_with_lowercase, y_train_bow_with_lowercase, X_test_bow_with_lowercase, y_test_bow_with_lowercase),
    "with_lemmatization": (X_train_bow_with_lemmatization, y_train_bow_with_lemmatization, X_test_bow_with_lemmatization, y_test_bow_with_lemmatization),
    "with_all": (X_train_bow_with_all, y_train_bow_with_all, X_test_bow_with_all, y_test_bow_with_all)
}

results_bow = {}
for name, (X_train, y_train, X_test, y_test) in datasets_bow.items():
    accuracy, cv_results, report = evaluate_model(X_train, y_train, X_test, y_test, best_params)
    results_bow[name] = {
        "accuracy": accuracy,
        "cv_results": cv_results,
        "classification_report": report
    }
    print(f"BoW Dataset: {name}, Test Accuracy: {accuracy}, Cross-Validation Mean Accuracy: {cv_results.mean()}, Std: {cv_results.std()}")
    print(f"Classification Report for BoW {name}:\n{report}\n")

# Afficher les résultats sous forme de DataFrame
results_bow_df = pd.DataFrame.from_dict(results_bow, orient='index')
print(results_bow_df)


BoW Dataset: combined, Test Accuracy: 0.9404990403071017, Cross-Validation Mean Accuracy: 0.9251026102195168, Std: 0.008562433508355205
Classification Report for BoW combined:
              precision    recall  f1-score   support

           0       0.97      0.91      0.94       264
           1       0.91      0.97      0.94       257

    accuracy                           0.94       521
   macro avg       0.94      0.94      0.94       521
weighted avg       0.94      0.94      0.94       521


BoW Dataset: without_stopwords, Test Accuracy: 0.9309021113243762, Cross-Validation Mean Accuracy: 0.9198256779192031, Std: 0.005192572267195257
Classification Report for BoW without_stopwords:
              precision    recall  f1-score   support

           0       0.96      0.91      0.93       264
           1       0.91      0.96      0.93       257

    accuracy                           0.93       521
   macro avg       0.93      0.93      0.93       521
weighted avg       0.93      0

#Classification avec le modèle SVM

##Recherche des meilleurs params

In [76]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

# Définir le classificateur
model = SVC(random_state=42)

# Définir la grille des hyperparamètres à rechercher
param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf'],
    'gamma': ['scale', 'auto']
}

# Définir les ensembles de données
datasets = {
    "TF-IDF": (X_train_tfidf_combined_unigram, y_train_tfidf_combined_unigram, X_test_tfidf_combined_unigram, y_test_tfidf_combined_unigram),
    "TF-IDF avec n-grammes": (X_train_tfidf_combined, y_train_tfidf_combined, X_test_tfidf_combined, y_test_tfidf_combined),
    "BoW": (X_train_bow_combined_unigram, y_train_bow_combined_unigram, X_test_bow_combined_unigram, y_test_bow_combined_unigram),
    "BoW avec n-grammes": (X_train_bow_combined, y_train_bow_combined, X_test_bow_combined, y_test_bow_combined_unigram)
}

# Rechercher les meilleurs hyperparamètres pour chaque ensemble de données
results = {}
for name, (X_train, y_train, X_test, y_test) in datasets.items():
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=5, scoring='accuracy', error_score=0)
    grid_search.fit(X_train, y_train)
    best_params = grid_search.best_params_
    best_score = grid_search.best_score_
    results[name] = {"best_params": best_params, "best_score": best_score}
    print(f"Meilleurs hyperparamètres pour {name}: {best_params}, Meilleur score: {best_score}")

# Assigner les meilleurs hyperparamètres à des variables individuelles
best_params_tfidf = results["TF-IDF"]["best_params"]
best_score_tfidf = results["TF-IDF"]["best_score"]

best_params_tfidf_ngrams = results["TF-IDF avec n-grammes"]["best_params"]
best_score_tfidf_ngrams = results["TF-IDF avec n-grammes"]["best_score"]

best_params_bow = results["BoW"]["best_params"]
best_score_bow =results["BoW"]["best_score"]

best_params_bow_ngrams = results["BoW avec n-grammes"]["best_params"]
best_score_bow_ngrams = results["BoW avec n-grammes"]["best_score"]

# Afficher les résultats des meilleurs hyperparamètres
print("\nMeilleurs hyperparamètres pour chaque ensemble de données :")
print("TF-IDF:", best_params_tfidf)
print("Score TF-IDF:", best_score_tfidf)
print("TF-IDF avec n-grammes:", best_params_tfidf_ngrams)
print("Score TF-IDF avec n-grammes:", best_score_tfidf_ngrams)
print("BoW:", best_params_bow)
print("Score BoW:", best_score_bow)
print("BoW avec n-grammes:", best_params_bow_ngrams)
print("Score BoW avec n-grammes:", best_score_bow_ngrams)



Meilleurs hyperparamètres pour TF-IDF: {'C': 10, 'gamma': 'scale', 'kernel': 'rbf'}, Meilleur score: 0.9913553772366722
Meilleurs hyperparamètres pour TF-IDF avec n-grammes: {'C': 10, 'gamma': 'scale', 'kernel': 'linear'}, Meilleur score: 0.9951969193875669
Meilleurs hyperparamètres pour BoW: {'C': 10, 'gamma': 'scale', 'kernel': 'rbf'}, Meilleur score: 0.9385399372809445
Meilleurs hyperparamètres pour BoW avec n-grammes: {'C': 10, 'gamma': 'scale', 'kernel': 'rbf'}, Meilleur score: 0.9414199409703008

Résultats:
TF-IDF: {'best_params': {'C': 10, 'gamma': 'scale', 'kernel': 'rbf'}, 'best_score': 0.9913553772366722}
TF-IDF avec n-grammes: {'best_params': {'C': 10, 'gamma': 'scale', 'kernel': 'linear'}, 'best_score': 0.9951969193875669}
BoW: {'best_params': {'C': 10, 'gamma': 'scale', 'kernel': 'rbf'}, 'best_score': 0.9385399372809445}
BoW avec n-grammes: {'best_params': {'C': 10, 'gamma': 'scale', 'kernel': 'rbf'}, 'best_score': 0.9414199409703008}


#Evaluation SVM

##Evaluation BOW ngrams

In [78]:
import pandas as pd
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, classification_report

# Définir la fonction pour évaluer le modèle
def evaluate_model(X_train, y_train, X_test, y_test, params):
    model = SVC(**params)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    cv_results = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy')
    report = classification_report(y_test, y_pred)

    return accuracy, cv_results, report

# Assigner les hyperparamètres optimaux trouvés précédemment
best_params_bow_ngrams = {'C': 10, 'gamma': 'scale', 'kernel': 'rbf'}

# Définir les ensembles de données
datasets_bow = {
    "combined": (X_train_bow_combined, y_train_bow_combined, X_test_bow_combined, y_test_bow_combined),
    "without_stopwords": (X_train_bow_without_stopwords, y_train_bow_without_stopwords, X_test_bow_without_stopwords, y_test_bow_without_stopwords),
    "with_lowercase": (X_train_bow_with_lowercase, y_train_bow_with_lowercase, X_test_bow_with_lowercase, y_test_bow_with_lowercase),
    "with_lemmatization": (X_train_bow_with_lemmatization, y_train_bow_with_lemmatization, X_test_bow_with_lemmatization, y_test_bow_with_lemmatization),
    "with_all": (X_train_bow_with_all, y_train_bow_with_all, X_test_bow_with_all, y_test_bow_with_all)
}

# Créer un dictionnaire pour stocker les résultats
results_bow = {}

# Utiliser les hyperparamètres best_params_bow_ngrams pour évaluer le modèle sur chaque ensemble de données
for name, (X_train, y_train, X_test, y_test) in datasets_bow.items():
    # Utiliser les hyperparamètres best_params_bow_ngrams pour cet ensemble de données
    best_params = best_params_bow_ngrams

    # Évaluer le modèle avec les meilleurs hyperparamètres sur cet ensemble de données
    accuracy, cv_results, report = evaluate_model(X_train, y_train, X_test, y_test, best_params)

    # Stocker les résultats dans le dictionnaire
    results_bow[name] = {
        "accuracy": accuracy,
        "cv_results": cv_results,
        "classification_report": report
    }

    # Afficher les résultats
    print(f"BoW Dataset: {name}, Test Accuracy: {accuracy}, Cross-Validation Mean Accuracy: {cv_results.mean()}, Std: {cv_results.std()}")
    print(f"Classification Report for BoW {name}:\n{report}\n")

# Afficher les résultats sous forme de DataFrame
results_bow_df = pd.DataFrame.from_dict(results_bow, orient='index')
print(results_bow_df)


BoW Dataset: combined, Test Accuracy: 0.9424184261036468, Cross-Validation Mean Accuracy: 0.9414199409703008, Std: 0.012722458661160292
Classification Report for BoW combined:
              precision    recall  f1-score   support

           0       0.98      0.90      0.94       264
           1       0.91      0.98      0.94       257

    accuracy                           0.94       521
   macro avg       0.95      0.94      0.94       521
weighted avg       0.95      0.94      0.94       521


BoW Dataset: without_stopwords, Test Accuracy: 0.9328214971209213, Cross-Validation Mean Accuracy: 0.9356645452868474, Std: 0.010374208496268985
Classification Report for BoW without_stopwords:
              precision    recall  f1-score   support

           0       0.97      0.90      0.93       264
           1       0.90      0.97      0.93       257

    accuracy                           0.93       521
   macro avg       0.93      0.93      0.93       521
weighted avg       0.94      0

##Evaluation BOW

In [79]:
import pandas as pd
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, classification_report

# Définir la fonction pour évaluer le modèle
def evaluate_model(X_train, y_train, X_test, y_test, params):
    model = SVC(**params)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    cv_results = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy')
    report = classification_report(y_test, y_pred)

    return accuracy, cv_results, report

# Assigner les hyperparamètres optimaux trouvés précédemment
best_params_bow_unigrams = {'C': 10, 'gamma': 'scale', 'kernel': 'rbf'}

# Définir les ensembles de données
datasets_bow = {
    "combined": (X_train_bow_combined_unigram, y_train_bow_combined_unigram, X_test_bow_combined_unigram, y_test_bow_combined_unigram),
    "without_stopwords": (X_train_bow_without_stopwords_unigram, y_train_bow_without_stopwords_unigram, X_test_bow_without_stopwords_unigram, y_test_bow_without_stopwords_unigram),
    "with_lowercase": (X_train_bow_with_lowercase_unigram, y_train_bow_with_lowercase_unigram, X_test_bow_with_lowercase_unigram, y_test_bow_with_lowercase_unigram),
    "with_lemmatization": (X_train_bow_with_lemmatization_unigram, y_train_bow_with_lemmatization_unigram, X_test_bow_with_lemmatization_unigram, y_test_bow_with_lemmatization_unigram),
    "with_all": (X_train_bow_with_all, y_train_bow_with_all, X_test_bow_with_all, y_test_bow_with_all)
}

# Créer un dictionnaire pour stocker les résultats
results_bow = {}

# Utiliser les hyperparamètres best_params_bow_unigrams pour évaluer le modèle sur chaque ensemble de données
for name, (X_train, y_train, X_test, y_test) in datasets_bow.items():
    # Utiliser les hyperparamètres best_params_bow_unigrams pour cet ensemble de données
    best_params = best_params_bow_unigrams

    # Évaluer le modèle avec les meilleurs hyperparamètres sur cet ensemble de données
    accuracy, cv_results, report = evaluate_model(X_train, y_train, X_test, y_test, best_params)

    # Stocker les résultats dans le dictionnaire
    results_bow[name] = {
        "accuracy": accuracy,
        "cv_results": cv_results,
        "classification_report": report
    }

    # Afficher les résultats
    print(f"BoW Dataset: {name}, Test Accuracy: {accuracy}, Cross-Validation Mean Accuracy: {cv_results.mean()}, Std: {cv_results.std()}")
    print(f"Classification Report for BoW {name}:\n{report}\n")

# Afficher les résultats sous forme de DataFrame
results_bow_df = pd.DataFrame.from_dict(results_bow, orient='index')
print(results_bow_df)


BoW Dataset: combined, Test Accuracy: 0.9500959692898272, Cross-Validation Mean Accuracy: 0.9385399372809445, Std: 0.013840986816448205
Classification Report for BoW combined:
              precision    recall  f1-score   support

           0       0.99      0.91      0.95       264
           1       0.91      0.99      0.95       257

    accuracy                           0.95       521
   macro avg       0.95      0.95      0.95       521
weighted avg       0.95      0.95      0.95       521


BoW Dataset: without_stopwords, Test Accuracy: 0.9481765834932822, Cross-Validation Mean Accuracy: 0.9385468548238333, Std: 0.010706039370025676
Classification Report for BoW without_stopwords:
              precision    recall  f1-score   support

           0       0.99      0.91      0.95       264
           1       0.91      0.99      0.95       257

    accuracy                           0.95       521
   macro avg       0.95      0.95      0.95       521
weighted avg       0.95      0

##Evaluation TFIDF

In [80]:
import pandas as pd
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, classification_report

# Définir la fonction pour évaluer le modèle
def evaluate_model(X_train, y_train, X_test, y_test, params):
    model = SVC(**params)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    cv_results = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy')
    report = classification_report(y_test, y_pred)

    return accuracy, cv_results, report

# Assigner les hyperparamètres optimaux trouvés précédemment
best_params_tfidf_unigrams = {'C': 10, 'gamma': 'scale', 'kernel': 'rbf'}

# Définir les ensembles de données TF-IDF
datasets_tfidf = {
    "combined": (X_train_tfidf_combined_unigram, y_train_tfidf_combined_unigram, X_test_tfidf_combined_unigram, y_test_tfidf_combined_unigram),
    "without_stopwords": (X_train_tfidf_without_stopwords_unigram, y_train_tfidf_without_stopwords_unigram, X_test_tfidf_without_stopwords_unigram, y_test_tfidf_without_stopwords_unigram),
    "with_lowercase": (X_train_tfidf_with_lowercase_unigram, y_train_tfidf_with_lowercase_unigram, X_test_tfidf_with_lowercase_unigram, y_test_tfidf_with_lowercase_unigram),
    "with_lemmatization": (X_train_tfidf_with_lemmatization_unigram, y_train_tfidf_with_lemmatization_unigram, X_test_tfidf_with_lemmatization_unigram, y_test_tfidf_with_lemmatization_unigram),
    "with_all": (X_train_tfidf_with_all_unigram, y_train_tfidf_with_all_unigram, X_test_tfidf_with_all_unigram, y_test_tfidf_with_all_unigram)
}

# Créer un dictionnaire pour stocker les résultats
results_tfidf = {}

# Utiliser les hyperparamètres best_params_tfidf_unigrams pour évaluer le modèle sur chaque ensemble de données TF-IDF
for name, (X_train, y_train, X_test, y_test) in datasets_tfidf.items():
    # Utiliser les hyperparamètres best_params_tfidf_unigrams pour cet ensemble de données
    best_params = best_params_tfidf_unigrams

    # Évaluer le modèle avec les meilleurs hyperparamètres sur cet ensemble de données
    accuracy, cv_results, report = evaluate_model(X_train, y_train, X_test, y_test, best_params)

    # Stocker les résultats dans le dictionnaire
    results_tfidf[name] = {
        "accuracy": accuracy,
        "cv_results": cv_results,
        "classification_report": report
    }

    # Afficher les résultats
    print(f"TF-IDF Dataset: {name}, Test Accuracy: {accuracy}, Cross-Validation Mean Accuracy: {cv_results.mean()}, Std: {cv_results.std()}")
    print(f"Classification Report for TF-IDF {name}:\n{report}\n")

# Afficher les résultats sous forme de DataFrame
results_tfidf_df = pd.DataFrame.from_dict(results_tfidf, orient='index')
print(results_tfidf_df)


TF-IDF Dataset: combined, Test Accuracy: 0.9942418426103646, Cross-Validation Mean Accuracy: 0.9913553772366722, Std: 0.006193750533461603
Classification Report for TF-IDF combined:
              precision    recall  f1-score   support

           0       1.00      0.99      0.99       264
           1       0.99      1.00      0.99       257

    accuracy                           0.99       521
   macro avg       0.99      0.99      0.99       521
weighted avg       0.99      0.99      0.99       521


TF-IDF Dataset: without_stopwords, Test Accuracy: 0.9942418426103646, Cross-Validation Mean Accuracy: 0.991838452315071, Std: 0.005390471099535014
Classification Report for TF-IDF without_stopwords:
              precision    recall  f1-score   support

           0       1.00      0.99      0.99       264
           1       0.99      1.00      0.99       257

    accuracy                           0.99       521
   macro avg       0.99      0.99      0.99       521
weighted avg       

##Evaluation TFIDF ngrams

In [82]:
import pandas as pd
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, classification_report

# Définir la fonction pour évaluer le modèle
def evaluate_model(X_train, y_train, X_test, y_test, params):
    model = SVC(**params)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    cv_results = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy')
    report = classification_report(y_test, y_pred)

    return accuracy, cv_results, report

# Assigner les hyperparamètres optimaux trouvés précédemment
best_params_tfidf_ngrams = {'C': 10, 'gamma': 'scale', 'kernel': 'linear'}

# Définir les ensembles de données TF-IDF
datasets_tfidf = {
    "combined": (X_train_tfidf_combined, y_train_tfidf_combined, X_test_tfidf_combined, y_test_tfidf_combined),
    "without_stopwords": (X_train_tfidf_without_stopwords, y_train_tfidf_without_stopwords, X_test_tfidf_without_stopwords, y_test_tfidf_without_stopwords),
    "with_lowercase": (X_train_tfidf_with_lowercase, y_train_tfidf_with_lowercase, X_test_tfidf_with_lowercase, y_test_tfidf_with_lowercase),
    "with_lemmatization": (X_train_tfidf_with_lemmatization, y_train_tfidf_with_lemmatization, X_test_tfidf_with_lemmatization, y_test_tfidf_with_lemmatization),
    "with_all": (X_train_tfidf_with_all, y_train_tfidf_with_all, X_test_tfidf_with_all, y_test_tfidf_with_all)
}

# Créer un dictionnaire pour stocker les résultats
results_tfidf = {}

# Utiliser les hyperparamètres best_params_tfidf_ngrams pour évaluer le modèle sur chaque ensemble de données TF-IDF
for name, (X_train, y_train, X_test, y_test) in datasets_tfidf.items():
    # Utiliser les hyperparamètres best_params_tfidf_ngrams pour cet ensemble de données
    best_params = best_params_tfidf_ngrams

    # Évaluer le modèle avec les meilleurs hyperparamètres sur cet ensemble de données
    accuracy, cv_results, report = evaluate_model(X_train, y_train, X_test, y_test, best_params)

    # Stocker les résultats dans le dictionnaire
    results_tfidf[name] = {
        "accuracy": accuracy,
        "cv_results": cv_results,
        "classification_report": report
    }

    # Afficher les résultats
    print(f"TF-IDF Dataset: {name}, Test Accuracy: {accuracy}, Cross-Validation Mean Accuracy: {cv_results.mean()}, Std: {cv_results.std()}")
    print(f"Classification Report for TF-IDF {name}:\n{report}\n")

# Afficher les résultats sous forme de DataFrame
results_tfidf_df = pd.DataFrame.from_dict(results_tfidf, orient='index')
print(results_tfidf_df)

TF-IDF Dataset: combined, Test Accuracy: 0.9942418426103646, Cross-Validation Mean Accuracy: 0.9951969193875669, Std: 0.0026353891577547144
Classification Report for TF-IDF combined:
              precision    recall  f1-score   support

           0       1.00      0.99      0.99       264
           1       0.99      1.00      0.99       257

    accuracy                           0.99       521
   macro avg       0.99      0.99      0.99       521
weighted avg       0.99      0.99      0.99       521


TF-IDF Dataset: without_stopwords, Test Accuracy: 0.9942418426103646, Cross-Validation Mean Accuracy: 0.9961561520014758, Std: 0.0032617565679631732
Classification Report for TF-IDF without_stopwords:
              precision    recall  f1-score   support

           0       1.00      0.99      0.99       264
           1       0.99      1.00      0.99       257

    accuracy                           0.99       521
   macro avg       0.99      0.99      0.99       521
weighted avg    